<a href="https://colab.research.google.com/github/mateus96mt/operational_research_second_work/blob/main/TRAB2_PO_MATEUS_TEIXEIRA_MAGALHAES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Problema modelo (https://www.gurobi.com/resource/food-manufacturing-i-and-ii/ (modelo i)):**
---


###Descrição resumida e traduzida:

Um produtor precisa refinar óleos pra produzir um produto alimentício comercializável. Os tipos de óleo necessários podem ser dividigos em 2 categorias:

| Categoria        | óleo         |
| ------------- |-------------| 
| Óleos vegetais:|  VEG 1<br>VEG 2 | 
| Óleos não-vegetais:     | OIL 1<br>OIL 2<br>OIL 3      |

O produtor pode escolher entre comprar os óleos no mês atual ou em meses futuros, o preço em dólares/tonelada para se comprar no mês atual ou nos seguintes é dado pela tabela:

| Mês |	VEG 1 |	VEG 2 |	OIL 1 |	OIL 2 |	OIL 3|
| ------------- |-------------| -------------| -------------| -------------| -------------| 
| Janeiro| 110 |	120 |	130 |	110 |	115|
| Fevereiro |130 |	130 |	110 |	90|	115|
| Março |110 |	140 |	130 |	100 |	95|
| Abril |120 |	110 |	120 |	120 |	125|
| Maio | 100 |	120 |	150 |	110 |	105|
| Junho |  	90 |	100 |	140 |	80| 	135 |

Outros fatores que devem ser levados em consideração:



1.   O produto final é vendido por $150 a tonelada.
2.   Cada categoria de óleo precisa ser refinada em uma refinaria diferente.
3.   A capacidade de refinamento, em qualquer mês, é fixa de 200 toneladas de óleo vegetal e 250 de óleo não-vegetal.
4.   Não existe perda no refinamento dos óleos, os óleos mandados para refinaria serão produzidos totalmente em óleos refinados.
5.   O custo para refinamento dos óleos pode ser ignorado.



Além das restrições acima existe uma restrição relacionada com a quantidade de óleos brutos (não-refinados) que pode ser armazedado para uso futuro, 1.000 toneladas para cada óleo a um custo de $5 por tonelada/mês. O produto alimentício produzido não pode ser armazenado nem os óleos refinados, somente os óleos brutos podem ser armazenados.

A última restrição é que o produto final produzido com os óleos deve ter um fator de "dureza" entre 3 e 6 em uma dada escala de "dureza". Considerando que a "dureza" é combinada de forma linear na produção do produto, temos a tabela a seguir:

|óleos |	"Dureza"|
| ------------- |-------------| 
|VEG 1 |	8.8|
|VEG 2 |	6.1|
|OIL 1 |	2.0|
|OIL2 |	4.2|
|OIL 3| 	5.0|

No início de Janeiro, existem 500 toneladas de cada tipo de óleo armazenado e para proposta do modelo essa também deve ser a quantidade armazenada no fim do mês de Junho.

**Objetivo:** descobrir os melhores meses para compra e manufatura de forma a maximizar os lucros.

Antes de fazermos modificações nos valores de entrada do modelo, vamos primeiro entender a modalegem matemática do problema original, isso porque:



*   A modelagem matemática do problema orignal pode ser utilizado no problema com valores de entrada alterados
*   Entendendo a modelagem matemática do problema original e sua solução podemos vislumbrar valores que impactam na solução do problema e gerar alterações que fazem mais sentido para um problema com valores modificados o que ajuda na verificação das diferenças e enálise de sensibilidade no modelo do problema com valores modificados



###Modelagem matemática:


**Conjuntos e índices:**

$t\in Meses = \{Jan, Fev, Mar, Abr, Mai, Jun\}$: conjunto de meses.

$V=\{VEG1, VEG2\}$:conjunto de óleos vegetais

$N = \{OIL1, OIL2,OIL3\}$:conjunto de óleos não-vegetais

$o \in \text{Oils} = V \cup N$: conjunto de óleos.

**Parâmetros:**

$\text{price} \in \mathbb{R}^+$: Preço de venda do produto.

$\text{init_store} \in \mathbb{R}^+$: Armazenamento inicial em toneladas.

$\text{target_store} \in \mathbb{R}^+$: Quantidade alvo a ser armazenada em toneladas.

$\text{holding_cost} \in \mathbb{R}^+$: Custo de armazenamento (dolar/ton/mês).

$\text{veg_cap} \in \mathbb{R}^+$: Capacidade de refinamento de óleo vegetal (ton/mês).

$\text{oil_cap} \in \mathbb{R}^+$: Capacidade de refinamento de óleo não-vegetal (ton/mês).

$\text{min_hardness} \in \mathbb{R}^+$: Dureza mínima permitida para produto final.

$\text{max_hardness} \in \mathbb{R}^+$: Dureza máxima permitida para produto final.

$\text{hardness}_o \in \mathbb{R}^+$: Dureza do óleo $o$.

$\text{cost}_{t,o} \in \mathbb{R}^+$: Custo estimado para compra de óleo $o$ no mês $t$.

**Variáveis de decisão:**

$\text{produce}_t \in \mathbb{R}^+$: Toneladas de comida produzida no mês $t$.

$\text{buy}_{t,o} \in \mathbb{R}^+$: Toneladas de óleo $o$ comprada no mês $t$.

$\text{consume}_{t,o} \in \mathbb{R}^+$: Toneladas de óleo $o$ usada no mês $t$.

$\text{store}_{t,o} \in \mathbb{R}^+$: Toneladas de óleo $o$ armazenada no mês $t$.


**Função objetivo:**

Maximizar o lucro total (dolar) no horizonte de planejamento:

\begin{equation}
\text{Maximize} \quad Z = \sum_{t \in \text{Months}}\text{price}*\text{produce}_t - \sum_{t \in \text{Months}}\sum_{o \in \text{Oils}}(\text{cost}_{t,o}*\text{consume}_{t,o} + \text{holding_cost}*\text{store}_{t,o})
\tag{0}
\end{equation}

**Restrições:**

- **Balanço inicial:** As toneladas de óleo $o$ compradas em janeiro somadas com a quantidade previamente armazenada deve ser igual a consumida e armazenada no mesmo mês.

\begin{equation}
\text{init_store} + \text{buy}_{Jan,o} = \text{consume}_{Jan,o} + \text{store}_{Jan,o} \quad \forall o \in \text{Oils}
\tag{1}
\end{equation}

- **Balanço:** As toneladas de óleo $o$ compradas no mês $t$ e as previamente armazenadas devem ser iguais as consumidas e armazenadas naquele mês.

\begin{equation}
\text{store}_{t-1,o} + \text{buy}_{t,o} = \text{consume}_{t,o} + \text{store}_{t,o} \quad \forall (t,o) \in \text{Meses} - \{\text{Jan}\} \times \text{Oils}
\tag{2}
\end{equation}

- **Objetivo de armazenagem final**: As toneladas de óleo $o$ armazenadas no fim do planejamento (fim de Junho) deve ser igual a almejada.

\begin{equation}
\text{store}_{Jun,o} = \text{target_store} \quad \forall o \in \text{Oils}
\tag{3}
\end{equation}

- **Capacidade de refinamento**: Quantidade de toneladas totais de óleo $o$ consumida no mês $t$ não pode exceder a capacidade de refinamento.

\begin{equation}
\sum_{o \in V}\text{consume}_{t,o} \leq \text{veg_cap} \quad \forall t \in \text{Meses}
\tag{4.1}
\end{equation}

\begin{equation}
\sum_{o \in N}\text{consume}_{t,o} \leq \text{oil_cap} \quad \forall t \in \text{Meses}
\tag{4.2}
\end{equation}

- **Dureza**: A dureza do produto produzido no mês $t$ deve respeitar a tolerância.

\begin{equation}
\text{min_hardness}*\text{produce}_t \leq \sum_{o \in \text{Oils}} \text{hardness}_o*\text{consume}_{t,o} \leq \text{max_hardness}*\text{produce}_t \quad \forall t \in \text{Meses}
\tag{5}
\end{equation}

- **Conservação de massa**: A quantidade total de toneladas de óleo consumidas no mês $t$ deve ser igual a quantidade total de comida produzida naquele mês.

\begin{equation}
\sum_{o \in \text{Oils}}\text{consume}_{t,o} = \text{produce}_t \quad \forall t \in \text{Months}
\tag{6}
\end{equation}

#**Solução do problema modelo usando Gurobi:**
---

Vamos agora implementar o modelo no python usando o Gurobi e resolve-lo.

###Instalando o Gurobi:

In [160]:
#Instalando a biblioteca do Gurobi
%pip install -i https://pypi.gurobi.com gurobipy

Looking in indexes: https://pypi.gurobi.com


###Importando Bibliotecas necessárias:




In [161]:
#Importando bibliotecas auxiliares
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

###Parâmetros originais do problema modelo:


In [162]:
# Parameters

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]

oils = ["VEG1", "VEG2", "OIL1", "OIL2", "OIL3"]

cost = {
    ('Jan', 'VEG1'): 110,
    ('Jan', 'VEG2'): 120,
    ('Jan', 'OIL1'): 130,
    ('Jan', 'OIL2'): 110,
    ('Jan', 'OIL3'): 115,
    ('Feb', 'VEG1'): 130,
    ('Feb', 'VEG2'): 130,
    ('Feb', 'OIL1'): 110,
    ('Feb', 'OIL2'): 90,
    ('Feb', 'OIL3'): 115,
    ('Mar', 'VEG1'): 110,
    ('Mar', 'VEG2'): 140,
    ('Mar', 'OIL1'): 130,
    ('Mar', 'OIL2'): 100,
    ('Mar', 'OIL3'): 95,
    ('Apr', 'VEG1'): 120,
    ('Apr', 'VEG2'): 110,
    ('Apr', 'OIL1'): 120,
    ('Apr', 'OIL2'): 120,
    ('Apr', 'OIL3'): 125,
    ('May', 'VEG1'): 100,
    ('May', 'VEG2'): 120,
    ('May', 'OIL1'): 150,
    ('May', 'OIL2'): 110,
    ('May', 'OIL3'): 105,
    ('Jun', 'VEG1'): 90,
    ('Jun', 'VEG2'): 100,
    ('Jun', 'OIL1'): 140,
    ('Jun', 'OIL2'): 80,
    ('Jun', 'OIL3'): 135
}


hardness = {"VEG1": 8.8, "VEG2": 6.1, "OIL1": 2.0, "OIL2": 4.2, "OIL3": 5.0}

price = 150
init_store = 500
target_store = 500
veg_cap = 200
oil_cap = 250

min_hardness = 3
max_hardness = 6
holding_cost = 5

###Variáveis de decisão:






In [163]:
food = gp.Model('Food Manufacture I')
# Quantity of food produced in each period
produce = food.addVars(months, name="Produce")
# Quantity bought of each product in each period
buy = food.addVars(months, oils, name = "Buy")
# Quantity used of each product  in each period
consume = food.addVars(months, oils, name = "Use")
# Quantity stored of each product  in each period
store = food.addVars(months, oils, name = "Store")

###Restrições:

In [164]:
#1. Initial Balance
Balance0 = food.addConstrs((init_store + buy[months[0], oil]
                 == consume[months[0], oil] + store[months[0], oil]
                 for oil in oils), "Initial_Balance")

#2. Balance
Balance = food.addConstrs((store[months[months.index(month)-1], oil] + buy[month, oil]
                 == consume[month, oil] + store[month, oil]
                 for oil in oils for month in months if month != month[0]), "Balance")

#3. Inventory Target
TargetInv = food.addConstrs((store[months[-1], oil] == target_store for oil in oils),"End_Balance")

#4.1 Vegetable Oil Capacity
VegCapacity = food.addConstrs((gp.quicksum(consume[month, oil] for oil in oils if "VEG" in oil)
                 <= veg_cap for month in months), "Capacity_Veg")

#4.2 Non-vegetable Oil Capacity
NonVegCapacity = food.addConstrs((gp.quicksum(consume[month, oil] for oil in oils if "OIL" in oil)
                 <= oil_cap for month in months), "Capacity_Oil")

#5. Hardness
HardnessMin = food.addConstrs((gp.quicksum(hardness[oil]*consume[month, oil] for oil in oils)
                 >= min_hardness*produce[month] for month in months), "Hardness_lower")
HardnessMax = food.addConstrs((gp.quicksum(hardness[oil]*consume[month, oil] for oil in oils)
                 <= max_hardness*produce[month] for month in months), "Hardness_upper")

#6. Mass Conservation
MassConservation = food.addConstrs((consume.sum(month) == produce[month] for month in months), "Mass_conservation")

###Função objetivo:

In [165]:
#0. Objective Function
obj = price*produce.sum() - buy.prod(cost) - holding_cost*store.sum()
food.setObjective(obj, GRB.MAXIMIZE) # maximize profit

###Otimizando o modelo com o Gurobi:


In [166]:
food.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 70 rows, 96 columns and 278 nonzeros
Model fingerprint: 0xd588eb19
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 5e+02]
Presolve removed 33 rows and 45 columns
Presolve time: 0.01s
Presolved: 37 rows, 51 columns, 149 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7375000e+05   1.703125e+03   0.000000e+00      0s
      32    1.0784259e+05   0.000000e+00   0.000000e+00      0s

Solved in 32 iterations and 0.04 seconds
Optimal objective  1.078425926e+05


##Analisando resultados:

###Quantidade de cada óleo que foi comprada em cada mês

In [167]:
rows = months.copy()
columns = oils.copy()
purchase_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, oil in buy.keys():
    if (abs(buy[month, oil].x) > 1e-6):
        purchase_plan.loc[month, oil] = np.round(buy[month, oil].x, 1)
purchase_plan

,VEG1,VEG2,OIL1,OIL2,OIL3
Jan,0.0,0.0,0.0,0.0,0.0
Feb,0.0,0.0,0.0,250.0,0.0
Mar,0.0,0.0,0.0,0.0,0.0
Apr,0.0,0.0,0.0,0.0,0.0
May,0.0,0.0,0.0,0.0,500.0
Jun,659.3,540.7,0.0,750.0,0.0


###Quantidade de óleo que foi consumida em cada mês:

In [168]:
rows = months.copy()
columns = oils.copy()
reqs = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, oil in consume.keys():
    if (abs(consume[month, oil].x) > 1e-6):
        reqs.loc[month, oil] = np.round(consume[month, oil].x, 1)
reqs

,VEG1,VEG2,OIL1,OIL2,OIL3
Jan,0.0,200.0,0.0,0.0,250.0
Feb,85.2,114.8,0.0,0.0,250.0
Mar,96.3,103.7,0.0,250.0,0.0
Apr,159.3,40.7,0.0,250.0,0.0
May,159.3,40.7,0.0,250.0,0.0
Jun,159.3,40.7,0.0,250.0,0.0


###Quantidade armazenada em cada mês:

In [169]:
rows = months.copy()
columns = oils.copy()
store_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, oil in store.keys():
    if (abs(store[month, oil].x) > 1e-6):
        store_plan.loc[month, oil] = np.round(store[month, oil].x, 1)
store_plan

,VEG1,VEG2,OIL1,OIL2,OIL3
Jan,500.0,300.0,500.0,500.0,250.0
Feb,414.8,185.2,500.0,750.0,0.0
Mar,318.5,81.5,500.0,500.0,0.0
Apr,159.3,40.7,500.0,250.0,0.0
May,0.0,0.0,500.0,0.0,500.0
Jun,500.0,500.0,500.0,500.0,500.0


###Quantidade de produto produzido em cada mês:

In [170]:
rows = months.copy()
columns = ["produce"]
production = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month in months:
    if (abs(produce[month].x) > 1e-6):
        production.loc[month, "produce"] = np.round(produce[month].x, 1)
production

,produce
Jan,450.0
Feb,450.0
Mar,450.0
Apr,450.0
May,450.0
Jun,450.0


Sabemos pelas restrições que a quantidade inicial armazenada em cada mês somada com a quantidade comprada - a quantidade comsunima é igual a quantidade armazenada no mês, facilmente podemos ver que os resultados respeitam essa restrição

###Verificação do teorema das folgas complementares: Variáveis duais, folgas, solução ótima, custo reduzido

Vamos observar primeiro cada uma das restriçoes e como elas foram definidas e verificarmos se o teorema das folgas complementares se aplica corretamente na solução ótima obtida

Seja $x^*$ viável para o primal e $y^*$ viável para o dual.

Ambas as soluções serão ótimas se, e somente se:

$y^*_{i}(b_i-\sum_{j=1}^{n}a_{ij}x^*_j)=0 \quad i=1,...m$

e

$(\sum_{i=1}^{m}a_{ij}y^*_i-c_j)x^*_j=0 \quad j = i...n$

In [200]:
"""
#1. Initial Balance
Balance0 = food.addConstrs((init_store + buy[months[0], oil]
                 == consume[months[0], oil] + store[months[0], oil]
                 for oil in oils), "Initial_Balance")
"""

#Essa restrição é na verdade uma lista de restrições, uma para cada óleo
#

# i = oil
# j = month

#primeira igualdade
print("         y_i*(b_i - sumatorio)         y_i*(Slack)")
for oil in oils:

  y_i = Balance0[oil].Pi
  
  b_i = consume[months[0], oil].X + store[months[0], oil].X

  sumatorio = init_store + buy[months[0], oil].X

  resultado = y_i*(b_i - sumatorio)

  print("i:", oils.index(oil), "     = ", resultado, "                     = ", y_i*Balance0[oil].Slack)

         y_i*(b_i - sumatorio)         y_i*(Slack)
i: 0      =  0.0                      =  0.0
i: 1      =  0.0                      =  0.0
i: 2      =  0.0                      =  0.0
i: 3      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0


In [201]:
"""
#2. Balance
Balance = food.addConstrs((store[months[months.index(month)-1], oil] + buy[month, oil]
                 == consume[month, oil] + store[month, oil]
                 for oil in oils for month in months if month != month[0]), "Balance")
"""

#O balanço é um conjunto de restrições, uma para cada óleo e para cada mês

# i = oil
# j = month

#primeira igualdade
print("         y_i*(b_i - sumatorio)         y_i*(Slack)")
for oil in oils:

  for month in months:

    if month != months[0]: 

      y_i = Balance[oil, month].Pi

      b_i = consume[month, oil].X + store[month, oil].X

      sumatorio = store[months[months.index(month)-1], oil].X + buy[month, oil].X

      resultado = y_i*(b_i - sumatorio)

      print("i:", oils.index(oil), ",j:", months.index(month), "     = ", resultado, "                     = ", y_i*Balance[oil, month].Slack)

         y_i*(b_i - sumatorio)         y_i*(Slack)
i: 0 ,j: 1      =  -0.0                      =  -0.0
i: 0 ,j: 2      =  -0.0                      =  -0.0
i: 0 ,j: 3      =  -0.0                      =  -0.0
i: 0 ,j: 4      =  -0.0                      =  -0.0
i: 0 ,j: 5      =  -0.0                      =  -0.0
i: 1 ,j: 1      =  -0.0                      =  -0.0
i: 1 ,j: 2      =  -0.0                      =  -0.0
i: 1 ,j: 3      =  -0.0                      =  -0.0
i: 1 ,j: 4      =  -4.050093593832571e-12                      =  -0.0
i: 1 ,j: 5      =  -0.0                      =  -0.0
i: 2 ,j: 1      =  -0.0                      =  -0.0
i: 2 ,j: 2      =  -0.0                      =  -0.0
i: 2 ,j: 3      =  -0.0                      =  -0.0
i: 2 ,j: 4      =  -0.0                      =  -0.0
i: 2 ,j: 5      =  -0.0                      =  -0.0
i: 3 ,j: 1      =  -0.0                      =  -0.0
i: 3 ,j: 2      =  -0.0                      =  -0.0
i: 3 ,j: 3      =  -0.0       

In [203]:
"""
#3. Inventory Target
TargetInv = food.addConstrs((store[months[-1], oil] == target_store for oil in oils),"End_Balance")
"""

#Lista de restrições definidas, uma para cada óleo

# i = oil
# j = month

#primeira igualdade
print("         y_i*(b_i - sumatorio)         y_i*(Slack)")
for oil in oils:

  y_i = TargetInv[oil].Pi
  
  b_i = target_store

  sumatorio = store[months[-1], oil].X

  resultado = y_i*(b_i - sumatorio)

  print("i:", oils.index(oil), "     = ", resultado, "                     = ", y_i*TargetInv[oil].Slack)

         y_i*(b_i - sumatorio)         y_i*(Slack)
i: 0      =  -0.0                      =  -0.0
i: 1      =  -0.0                      =  -0.0
i: 2      =  -0.0                      =  -0.0
i: 3      =  0.0                      =  0.0
i: 4      =  -0.0                      =  -0.0


In [205]:
"""
#4.1 Vegetable Oil Capacity
VegCapacity = food.addConstrs((gp.quicksum(consume[month, oil] for oil in oils if "VEG" in oil)
                 <= veg_cap for month in months), "Capacity_Veg")
"""

#Uma lista de restrições, uma restrição para cada mês
# i = month
# j = oil

#primeira igualdade
print("         y_i*(b_i - sumatorio)         y_i*(Slack)")
for month in months:

  y_i = VegCapacity[month].Pi
  
  b_i = veg_cap

  sumatorio = gp.quicksum(consume[month, oil].X for oil in oils if "VEG" in oil).getValue()

  resultado = y_i*(b_i - sumatorio)

  print("i:", oils.index(oil), "     = ", resultado, "                     = ", y_i*VegCapacity[month].Slack)

         y_i*(b_i - sumatorio)         y_i*(Slack)
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0


In [206]:
"""
#4.2 Non-vegetable Oil Capacity
NonVegCapacity = food.addConstrs((gp.quicksum(consume[month, oil] for oil in oils if "OIL" in oil)
                 <= oil_cap for month in months), "Capacity_Oil")
"""

#Uma restrição para cada mês
# i = month
# j = oil

#primeira igualdade
print("         y_i*(b_i - sumatorio)         y_i*(Slack)")
for month in months:

  y_i = NonVegCapacity[month].Pi
  
  b_i = oil_cap

  sumatorio = gp.quicksum(consume[month, oil].X for oil in oils if "OIL" in oil).getValue()

  resultado = y_i*(b_i - sumatorio)

  print("i:", oils.index(oil), "     = ", resultado, "                     = ", y_i*NonVegCapacity[month].Slack)

         y_i*(b_i - sumatorio)         y_i*(Slack)
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0


In [207]:
"""
#5. Hardness
HardnessMin = food.addConstrs((gp.quicksum(hardness[oil]*consume[month, oil] for oil in oils)
                 >= min_hardness*produce[month] for month in months), "Hardness_lower")

HardnessMax = food.addConstrs((gp.quicksum(hardness[oil]*consume[month, oil] for oil in oils)
                 <= max_hardness*produce[month] for month in months), "Hardness_upper")
"""

#As duas restrições são uma lista de restrições para cada mês

# i = month
# j = oil

#primeira igualdade
print("         y_i*(b_i - sumatorio)         y_i*(Slack)")
for month in months:

  y_i = HardnessMin[month].Pi
  
  b_i = min_hardness*produce[month].X

  sumatorio = gp.quicksum(hardness[oil]*consume[month, oil].X for oil in oils).getValue()

  resultado = y_i*(b_i - sumatorio)

  print("i:", oils.index(oil), "     = ", resultado, "                     = ", y_i*HardnessMin[month].Slack)


print("         y_i*(b_i - sumatorio)         y_i*(Slack)")
for month in months:

  y_i = HardnessMax[month].Pi
  
  b_i = max_hardness*produce[month].X

  sumatorio = gp.quicksum(hardness[oil]*consume[month, oil].X for oil in oils).getValue()

  resultado = y_i*(b_i - sumatorio)

  print("i:", oils.index(oil), "     = ", resultado, "                     = ", y_i*HardnessMax[month].Slack)

         y_i*(b_i - sumatorio)         y_i*(Slack)
i: 4      =  -0.0                      =  -0.0
i: 4      =  -0.0                      =  -0.0
i: 4      =  -0.0                      =  -0.0
i: 4      =  -0.0                      =  -0.0
i: 4      =  -0.0                      =  -0.0
i: 4      =  -0.0                      =  -0.0
         y_i*(b_i - sumatorio)         y_i*(Slack)
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0


In [208]:
"""
#6. Mass Conservation
MassConservation = food.addConstrs((consume.sum(month) == produce[month] for month in months), "Mass_conservation")
"""

#Uma restrição para cada mês

# i = month
# j = oil

#primeira igualdade
print("         y_i*(b_i - sumatorio)         y_i*(Slack)")
for month in months:

  y_i = MassConservation[month].Pi
  
  b_i = produce[month].X

  sumatorio = consume.sum(month).getValue()

  resultado = y_i*(b_i - sumatorio)

  print("i:", oils.index(oil), "     = ", resultado, "                     = ", y_i*MassConservation[month].Slack)

         y_i*(b_i - sumatorio)         y_i*(Slack)
i: 4      =  -0.0                      =  -0.0
i: 4      =  -0.0                      =  -0.0
i: 4      =  -0.0                      =  -0.0
i: 4      =  -0.0                      =  -0.0
i: 4      =  -0.0                      =  -0.0
i: 4      =  -0.0                      =  -0.0


#**Problema modelo com parâmetros modificados:**
---


###**OBSERVAÇÃO (RESTRIÇÃO DE LIMITE DE ARMAZENAMENTO DE ÓLEO POR MÊS):**

No problema modelo esqueceram de impor o limite de armazenamento de óleo por mês, $1.000$ toneladas de cada óleo por mês seria o limite.

Como no modelo exemplo foi esquecido de inserir essa restrição vamos desconsidera-la, nesse caso não haverá limite de armazenamento por mês, uma vez que essa restrição não foi imposta no problema modelo.

##Analisando parâmetros do problema modelo

Vamos definir alguns parâmetros para serem modificados em cima dos parâmetros do problema modelo.

**Preço de venda do produto "price":**

O parâmetro "price" só aparece na função objetivo:

\begin{equation}
\text{Maximize} \quad Z = \sum_{t \in \text{Months}}\color{red}{price}*\text{produce}_t - \sum_{t \in \text{Months}}\sum_{o \in \text{Oils}}(\text{cost}_{t,o}*\text{consume}_{t,o} + \text{holding_cost}*\text{store}_{t,o})
\tag{0}
\end{equation}

Diminuir o valor de "price" teria um impacto direto na função objetivo e no valor de "produce", pois mais comida deverá ser produzida para gerar o mesmo lucro e isso impacta diretamente os resultados.



**Capacidades de refinamento de cada óleo, "veg_cap" e "oil_cap":**




\begin{equation}
\sum_{o \in V}\text{consume}_{t,o} \leq \color{red}{veg\_cap} \quad \forall t \in \text{Meses}
\tag{4.1}
\end{equation}

\begin{equation}
\sum_{o \in N}\text{consume}_{t,o} \leq \color{red}{oil\_cap} \quad \forall t \in \text{Meses}
\tag{4.2}
\end{equation}

Ao reduzir as capacidades de refinamento temos que as capacidades de consumo (que é de óleo refinado, que não pode ser armazenado) serão afetadas e deverão ser reduzidas também, consequentemente o valor das quantidades compradas também será resuzida, ou, se não for reduzida terá de ser armazenada para refinamento posterior em maior quantidade uma vez que a capacidade de refinamento diminuiu.

**Quantidade inicial armazenada "init_store":**


Aparece somente em uma restrição:

\begin{equation}
\color{red}{init\_store} + \text{buy}_{Jan,o} = \text{consume}_{Jan,o} + \text{store}_{Jan,o} \quad \forall o \in \text{Oils}
\tag{1}
\end{equation}

O "init_store" vai ter um impacto maior no primeiro mês, "Janeiro".

Alterar o valor de "init_store" vai fazer com que os valores de "buy", "consume" e "store" sejam alterados para balancear a igualdade.

Aumentando o "init_store" o lado esquerdo da equação vai ser aumentado fazendo com que o valor de "buy", ou seja, a quantidade comprada diminua e os valores consumidos "consume" e armazenados "store" aumentem par ao mês de Janeiro.


**Quantidades mínimas e máximas de "dureza", "min_hardness" e "max_hardness":**

O valor de dureza mínima e máxima, "min_hardness" e "max_hardness", respectivamente, aparecem somente na restrição:

\begin{equation}
\color{red}{min\_hardness}*\text{produce}_t \leq \sum_{o \in \text{Oils}} \text{hardness}_o*\text{consume}_{t,o} \leq \color{red}{max\_hardness}*\text{produce}_t \quad \forall t \in \text{Meses}
\tag{5}
\end{equation}

Alterar esses valores pode criar uma inviabilidade no problema, por exemplo colocando um valor mínimo e máximo menor do que o menor valor existente na tabela de dureza do óleos, dessa forma não vai existir combinação (dado que os valores são positivos) que gere um produto com uma dureza numa faixa de dureza tão baixa. Se a faixa de dureza for extendida pode gerar inviabilidade tamém dado as demais restrições 

**Dureza dos óleos, "hardness":**

Aparece na restrição anterior:

\begin{equation}
\text{min_hardness}*\text{produce}_t \leq \sum_{o \in \text{Oils}} \color{red}{hardness}_o*\text{consume}_{t,o} \leq \text{max_hardness}*\text{produce}_t \quad \forall t \in \text{Meses}
\tag{5}
\end{equation}

Altera-la também pode gerar iviabilidade dependendo dos valores de dureza máxima e mínima

**Custo de armazenamento:**

Aparece também na função objetivo:

\begin{equation}
\text{Maximize} \quad Z = \sum_{t \in \text{Months}}\text{price}*\text{produce}_t - \sum_{t \in \text{Months}}\sum_{o \in \text{Oils}}(\text{cost}_{t,o}*\text{consume}_{t,o} + \color{red}{holding\_cost}*\text{store}_{t,o})
\tag{0}
\end{equation}

Como ele aparece de forma negativa na função objetivo que é de maximização e aparece multiplicado por "store", aumentar o custo de armazenamento tenderia a diminuir a quantidade armazenada, "store".

**Quantidade alvo no fim do período, "target_store":**

Aparece na restrição do último mês:

\begin{equation}
\text{store}_{Jun,o} = \color{red}{target\_store} \quad \forall o \in \text{Oils}
\tag{3}
\end{equation}

Impacta mais os valores do último mês e faz com que a quantidade armazenada seja sempre o valor dado no último mês.

**Custo de compra de óleo no mês, "cost":**

O custo de comprar o óleo, "cost" aparece na função objetivo:

\begin{equation}
\text{Maximize} \quad Z = \sum_{t \in \text{Months}}\text{price}*\text{produce}_t - \sum_{t \in \text{Months}}\sum_{o \in \text{Oils}}(\color{red}{cost}_{t,o}*\text{consume}_{t,o} + \text{holding_cost}*\text{store}_{t,o})
\tag{0}
\end{equation}

Aparece de forma negativa na função objetivo de maximização multiplicado por "consume". Ao aumentar o custo tende-se que a solução ótima ai ter quantidades consumidas menores

##Alteração dos parâmetros do problema

Vamos alterar os parâmetros com base no que foi analisado anteriormente.

Vamos diminuir o preço de venda para que a quantidade produzida tenha de aumentar para compensar os ganhos.

"price": de 150 para 50

In [209]:
price = 50

Para que mais quantidade possa ser produzida vamos aumentar as capacidades de refinamento:

"veg_cap": de 200 -> 400
"oil_cap": de 250 -> 500

In [210]:
veg_cap = 400
oil_cap = 500

Vamos aumentar a quantidade inicial armazenada:

"init_store": 500 para 800

In [211]:
init_store = 800

Vamos aumentar "min_hardness" e manter "max_hardness" para tentar favorecer a compra de óleos com dureza maior.

"min_hardness": 3 para 5.5
"max_hardness": manter valor de 6.0

In [212]:
min_hardness = 5.5
max_hardness = 6

Vamos alterar os valores de "hardness":

De:

|Oils |	Hardness|
| ------------- |-------------| 
|VEG 1 |	8.8|
|VEG 2 |	6.1|
|OIL 1 |	2.0|
|OIL2 |	4.2|
|OIL 3| 	5.0|

Para:

|Oils |	Hardness|
| ------------- |-------------| 
|VEG 1 |	2.0|
|VEG 2 |	4.2|
|OIL 1 |	8.8|
|OIL2 |	6.1|
|OIL 3| 	5.0|

Dessa forma os óleos OIL 1 e OIL 2 vão ser favorecidos pelas restrições de "hardness" alterada anteriormente

In [213]:
hardness = {"VEG1": 2.0, "VEG2": 4.2, "OIL1": 8.8, "OIL2": 6.1, "OIL3": 5.0}

Vamos alterar o custo de armazenamento, reduzindo o mesmo para favorecer armazenarmos mais quantidade de óleos:

"holding_cost": de 5 para 1

In [214]:
holding_cost = 1

A quantidade armazenada no último mês deve ser igual a quantidade armazenada inicialmente, por definição do problema modelo.

"target_store": de 500 para 800

In [215]:
target_store = 800

Vamos alterar os custos de compra de forma a diminuir os custos no geral para favorecer mais compras e penalizar um produto em específico, vamos penalizar o óleo.

Vamos penalizar "OIL1", uma vez que beneficiamos ele aumentando o valor mínimo de dureza favorecento sua escolha, porém vamos agora penaliza-lo severamente em seu custo, esperamos ver ele aparecer menos na solução ótima.

In [216]:
cost = {
    ('Jan', 'VEG1'): 30,
    ('Jan', 'VEG2'): 50,
    ('Jan', 'OIL1'): 150,
    ('Jan', 'OIL2'): 60,
    ('Jan', 'OIL3'): 40,
    ('Feb', 'VEG1'): 55,
    ('Feb', 'VEG2'): 45,
    ('Feb', 'OIL1'): 180,
    ('Feb', 'OIL2'): 90,
    ('Feb', 'OIL3'): 70,
    ('Mar', 'VEG1'): 40,
    ('Mar', 'VEG2'): 30,
    ('Mar', 'OIL1'): 190,
    ('Mar', 'OIL2'): 40,
    ('Mar', 'OIL3'): 60,
    ('Apr', 'VEG1'): 75,
    ('Apr', 'VEG2'): 65,
    ('Apr', 'OIL1'): 165,
    ('Apr', 'OIL2'): 55,
    ('Apr', 'OIL3'): 45,
    ('May', 'VEG1'): 65,
    ('May', 'VEG2'): 55,
    ('May', 'OIL1'): 155,
    ('May', 'OIL2'): 75,
    ('May', 'OIL3'): 60,
    ('Jun', 'VEG1'): 75,
    ('Jun', 'VEG2'): 65,
    ('Jun', 'OIL1'): 145,
    ('Jun', 'OIL2'): 50,
    ('Jun', 'OIL3'): 65
}

Conjunto de meses e óleos são os mesmos.



In [217]:
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]

oils = ["VEG1", "VEG2", "OIL1", "OIL2", "OIL3"]

##Implementação do modelo e execução

###Instalando o Gurobi


In [218]:
#Instalando a biblioteca do Gurobi
%pip install -i https://pypi.gurobi.com gurobipy

Looking in indexes: https://pypi.gurobi.com


###Importando bibliotecas necessárias

In [219]:
#Importando bibliotecas auxiliares
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

###Implementação e variáveis de decisão

In [220]:
food = gp.Model('Food Manufacture I')
# Quantity of food produced in each period
produce = food.addVars(months, name="Produce")
# Quantity bought of each product in each period
buy = food.addVars(months, oils, name = "Buy")
# Quantity used of each product  in each period
consume = food.addVars(months, oils, name = "Use")
# Quantity stored of each product  in each period
store = food.addVars(months, oils, name = "Store")

###Restrições

In [221]:
#1. Initial Balance
Balance0 = food.addConstrs((init_store + buy[months[0], oil]
                 == consume[months[0], oil] + store[months[0], oil]
                 for oil in oils), "Initial_Balance")

#2. Balance
Balance = food.addConstrs((store[months[months.index(month)-1], oil] + buy[month, oil]
                 == consume[month, oil] + store[month, oil]
                 for oil in oils for month in months if month != month[0]), "Balance")

#3. Inventory Target
TargetInv = food.addConstrs((store[months[-1], oil] == target_store for oil in oils),"End_Balance")

#4.1 Vegetable Oil Capacity
VegCapacity = food.addConstrs((gp.quicksum(consume[month, oil] for oil in oils if "VEG" in oil)
                 <= veg_cap for month in months), "Capacity_Veg")

#4.2 Non-vegetable Oil Capacity
NonVegCapacity = food.addConstrs((gp.quicksum(consume[month, oil] for oil in oils if "OIL" in oil)
                 <= oil_cap for month in months), "Capacity_Oil")

#5. Hardness
HardnessMin = food.addConstrs((gp.quicksum(hardness[oil]*consume[month, oil] for oil in oils)
                 >= min_hardness*produce[month] for month in months), "Hardness_lower")
HardnessMax = food.addConstrs((gp.quicksum(hardness[oil]*consume[month, oil] for oil in oils)
                 <= max_hardness*produce[month] for month in months), "Hardness_upper")

#6. Mass Conservation
MassConservation = food.addConstrs((consume.sum(month) == produce[month] for month in months), "Mass_conservation")

###Função objetivo

In [222]:
#0. Objective Function
obj = price*produce.sum() - buy.prod(cost) - holding_cost*store.sum()
food.setObjective(obj, GRB.MAXIMIZE) # maximize profit

###Otimizando o modelo com o Gurobi

In [223]:
food.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 70 rows, 96 columns and 278 nonzeros
Model fingerprint: 0x99a08763
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 8e+02]
Presolve removed 33 rows and 50 columns
Presolve time: 0.01s
Presolved: 37 rows, 46 columns, 144 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5010160e+05   1.856250e+03   0.000000e+00      0s
      42    2.7546154e+04   0.000000e+00   0.000000e+00      0s

Solved in 42 iterations and 0.02 seconds
Optimal objective  2.754615385e+04


###Quantidade comprada de cada óleo em cada mês

In [224]:
rows = months.copy()
columns = oils.copy()
purchase_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, oil in buy.keys():
    if (abs(buy[month, oil].x) > 1e-6):
        purchase_plan.loc[month, oil] = np.round(buy[month, oil].x, 1)
purchase_plan

,VEG1,VEG2,OIL1,OIL2,OIL3
Jan,0.0,0.0,0.0,0.0,200.0
Feb,0.0,0.0,0.0,0.0,0.0
Mar,0.0,1215.4,0.0,2800.0,0.0
Apr,0.0,0.0,0.0,0.0,0.0
May,0.0,0.0,0.0,0.0,0.0
Jun,0.0,0.0,0.0,0.0,0.0


###Quantidade de cada óleo consumida em cada mês

In [225]:
rows = months.copy()
columns = oils.copy()
reqs = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, oil in consume.keys():
    if (abs(consume[month, oil].x) > 1e-6):
        reqs.loc[month, oil] = np.round(consume[month, oil].x, 1)
reqs

,VEG1,VEG2,OIL1,OIL2,OIL3
Jan,0.0,230.8,0.0,500.0,0.0
Feb,0.0,61.5,0.0,300.0,200.0
Mar,0.0,230.8,0.0,500.0,0.0
Apr,0.0,230.8,0.0,500.0,0.0
May,0.0,230.8,0.0,500.0,0.0
Jun,0.0,230.8,0.0,500.0,0.0


###Quantidade de cada óleo armazenada em cada mês

In [226]:
rows = months.copy()
columns = oils.copy()
store_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, oil in store.keys():
    if (abs(store[month, oil].x) > 1e-6):
        store_plan.loc[month, oil] = np.round(store[month, oil].x, 1)
store_plan

,VEG1,VEG2,OIL1,OIL2,OIL3
Jan,800.0,569.2,800.0,300.0,1000.0
Feb,800.0,507.7,800.0,0.0,800.0
Mar,800.0,1492.3,800.0,2300.0,800.0
Apr,800.0,1261.5,800.0,1800.0,800.0
May,800.0,1030.8,800.0,1300.0,800.0
Jun,800.0,800.0,800.0,800.0,800.0


###Quantidade de produto produzido em cada mês:

In [227]:
rows = months.copy()
columns = ["produce"]
production = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month in months:
    if (abs(produce[month].x) > 1e-6):
        production.loc[month, "produce"] = np.round(produce[month].x, 1)
production

,produce
Jan,730.8
Feb,561.5
Mar,730.8
Apr,730.8
May,730.8
Jun,730.8


##Análise sobre os resultados do modelo com parâmetros alterados:

###Discussão acerca dos resultados

A primeira alteração sugerida na etapa de "Analisando parâmetros do problema modelo" foi de reduzir o valor de venda do produto produzido e o esperado, conforme visto, seria que a quantidade produzida deveria almentar bastante para compensar essa diminuiçao no preço de venda e manter o lucro (função objetivo).

Vemos que isso aconteceu, basta comparar as tabelas em "Quantidade de produto produzido em cada mês:" do modelo com parâmetros originais e do modelo com parâmetros alterados.

Aumentando as capacidades de refinamento de ambos os óleos vemos que as capacidades consumidas (refinadas) aumentaram conforme o esperado, refletindo o aumento da quantidade de produto produzido que é a soma de todos os óleos consumidos em cada mês.


Aumentando a capacidade de armazenamento inicial vemos que aumentaram as quantidade armazenadas no mês inicial, conforme esperado.

Aumentando a dureza mínima e mantendo a dureza máxima vemos que os óleos com maior dureza foram priorizados como o esperado, nesse caso, a compra do OIL2, apesar do OIL1 ter maior dureza ele foi penalizado no preço, conforme foi alterado a tabela de preço para penaliza-lo na alteração dos parâmetros.

Os valores de dureza "hardness" foram aumentados para favorecer o OIL2 e OIL1 e vemos que teve o efeito esperado.

O custo de armazenamento foi reduzido o que aumentou significamente as quantidades armazenadas para cada óleo, conforme esperado.

O "target_store" definido pra 800 fez com que no final de Junho as quantidades armazenadas fossem exatamente 800 conforme imposto por restrição.

O custo "cost" influencia na quantidade consumida conforme visto anteriormente na análise dos parâmetros, a alteração de diminuir todos os custos no geral e aumentar somente os custos do OIL1 fez com que fosse severamente penalizado, basta vem a tabela de compra, consumo e armazenamento.

O óleo OIL1 foi tão penalizado que não foi comprado nem uma única vez, nem consumido (utilizado para gerar produtos), todo o estoque inicial do mês de Janeiro foi mantido até o fim de Junho.

###Recursos econômicamente significantes (explicação para cliente):



Os 3 recursos mais significantes são a capacidade de refinamento de óleo vegetal, "veg_cap" e não-vegetal, "oil_cap", o preço de venda do produto, "price".

Se o preço de venda do produto for muito baixo será necessário produzir uma quantidade muito maior para vender e manter o lucro e para produzirmos em maior quantidade é necessário que as capacidades de refinamento sejam maiores também.

Se o preço de venda diminuir e as capacidades forem mantidas será muito difícil manter o lucro e a solução ótima para o problema terá um lucro menor.

Deve então haver um equilíbrio entre o preço de venda e as capacidades de produção de forma que o lucro seja mantido.

Caso deseje diminuir o preço de venda do produto para deixa-lo mais competivivo no mercado, tenha em mente que será necessário vender mais para manter o lucro e consequentemente produzir mais e para isso as capacidades de refinamento dos óleos deve aumentar ou o negócio se torna insustentável, lucro baixos que podem inviabilizar o modelo de negócio.

###Verificação do teorema das folgas complementares: Variáveis duais, folgas, solução ótima, custo reduzido

Vamos observar primeiro cada uma das restriçoes e como elas foram definidas e verificarmos se o teorema das folgas complementares se aplica corretamente na solução ótima obtida

Seja $x^*$ viável para o primal e $y^*$ viável para o dual.

Ambas as soluções serão ótimas se, e somente se:

$y^*_{i}(b_i-\sum_{j=1}^{n}a_{ij}x^*_j)=0 \quad i=1,...m$

e

$(\sum_{i=1}^{m}a_{ij}y^*_i-c_j)x^*_j=0 \quad j = i...n$

In [228]:
"""
#1. Initial Balance
Balance0 = food.addConstrs((init_store + buy[months[0], oil]
                 == consume[months[0], oil] + store[months[0], oil]
                 for oil in oils), "Initial_Balance")
"""

#Essa restrição é na verdade uma lista de restrições, uma para cada óleo
#

# i = oil
# j = month

#primeira igualdade
print("         y_i*(b_i - sumatorio)         y_i*(Slack)")
for oil in oils:

  y_i = Balance0[oil].Pi
  
  b_i = consume[months[0], oil].X + store[months[0], oil].X

  sumatorio = init_store + buy[months[0], oil].X

  resultado = y_i*(b_i - sumatorio)

  print("i:", oils.index(oil), "     = ", resultado, "                     = ", y_i*Balance0[oil].Slack)

         y_i*(b_i - sumatorio)         y_i*(Slack)
i: 0      =  0.0                      =  0.0
i: 1      =  0.0                      =  0.0
i: 2      =  0.0                      =  0.0
i: 3      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0


In [229]:
"""
#2. Balance
Balance = food.addConstrs((store[months[months.index(month)-1], oil] + buy[month, oil]
                 == consume[month, oil] + store[month, oil]
                 for oil in oils for month in months if month != month[0]), "Balance")
"""

#O balanço é um conjunto de restrições, uma para cada óleo e para cada mês

# i = oil
# j = month

#primeira igualdade
print("         y_i*(b_i - sumatorio)         y_i*(Slack)")
for oil in oils:

  for month in months:

    if month != months[0]: 

      y_i = Balance[oil, month].Pi

      b_i = consume[month, oil].X + store[month, oil].X

      sumatorio = store[months[months.index(month)-1], oil].X + buy[month, oil].X

      resultado = y_i*(b_i - sumatorio)

      print("i:", oils.index(oil), ",j:", months.index(month), "     = ", resultado, "                     = ", y_i*Balance[oil, month].Slack)

         y_i*(b_i - sumatorio)         y_i*(Slack)
i: 0 ,j: 1      =  -0.0                      =  -0.0
i: 0 ,j: 2      =  -0.0                      =  -0.0
i: 0 ,j: 3      =  -0.0                      =  -0.0
i: 0 ,j: 4      =  -0.0                      =  -0.0
i: 0 ,j: 5      =  -0.0                      =  -0.0
i: 1 ,j: 1      =  -0.0                      =  -0.0
i: 1 ,j: 2      =  -0.0                      =  -0.0
i: 1 ,j: 3      =  -0.0                      =  -0.0
i: 1 ,j: 4      =  -0.0                      =  -0.0
i: 1 ,j: 5      =  -0.0                      =  -0.0
i: 2 ,j: 1      =  -0.0                      =  -0.0
i: 2 ,j: 2      =  -0.0                      =  -0.0
i: 2 ,j: 3      =  -0.0                      =  -0.0
i: 2 ,j: 4      =  -0.0                      =  -0.0
i: 2 ,j: 5      =  -0.0                      =  -0.0
i: 3 ,j: 1      =  -0.0                      =  -0.0
i: 3 ,j: 2      =  -0.0                      =  -0.0
i: 3 ,j: 3      =  -0.0                      =  

In [230]:
"""
#3. Inventory Target
TargetInv = food.addConstrs((store[months[-1], oil] == target_store for oil in oils),"End_Balance")
"""

#Lista de restrições definidas, uma para cada óleo

# i = oil
# j = month

#primeira igualdade
print("         y_i*(b_i - sumatorio)         y_i*(Slack)")
for oil in oils:

  y_i = TargetInv[oil].Pi
  
  b_i = target_store

  sumatorio = store[months[-1], oil].X

  resultado = y_i*(b_i - sumatorio)

  print("i:", oils.index(oil), "     = ", resultado, "                     = ", y_i*TargetInv[oil].Slack)

         y_i*(b_i - sumatorio)         y_i*(Slack)
i: 0      =  -0.0                      =  -0.0
i: 1      =  -0.0                      =  -0.0
i: 2      =  -0.0                      =  -0.0
i: 3      =  0.0                      =  0.0
i: 4      =  -0.0                      =  -0.0


In [231]:
"""
#4.1 Vegetable Oil Capacity
VegCapacity = food.addConstrs((gp.quicksum(consume[month, oil] for oil in oils if "VEG" in oil)
                 <= veg_cap for month in months), "Capacity_Veg")
"""

#Uma lista de restrições, uma restrição para cada mês
# i = month
# j = oil

#primeira igualdade
print("         y_i*(b_i - sumatorio)         y_i*(Slack)")
for month in months:

  y_i = VegCapacity[month].Pi
  
  b_i = veg_cap

  sumatorio = gp.quicksum(consume[month, oil].X for oil in oils if "VEG" in oil).getValue()

  resultado = y_i*(b_i - sumatorio)

  print("i:", oils.index(oil), "     = ", resultado, "                     = ", y_i*VegCapacity[month].Slack)

         y_i*(b_i - sumatorio)         y_i*(Slack)
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0


In [232]:
"""
#4.2 Non-vegetable Oil Capacity
NonVegCapacity = food.addConstrs((gp.quicksum(consume[month, oil] for oil in oils if "OIL" in oil)
                 <= oil_cap for month in months), "Capacity_Oil")
"""

#Uma restrição para cada mês
# i = month
# j = oil

#primeira igualdade
print("         y_i*(b_i - sumatorio)         y_i*(Slack)")
for month in months:

  y_i = NonVegCapacity[month].Pi
  
  b_i = oil_cap

  sumatorio = gp.quicksum(consume[month, oil].X for oil in oils if "OIL" in oil).getValue()

  resultado = y_i*(b_i - sumatorio)

  print("i:", oils.index(oil), "     = ", resultado, "                     = ", y_i*NonVegCapacity[month].Slack)

         y_i*(b_i - sumatorio)         y_i*(Slack)
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0


In [233]:
"""
#5. Hardness
HardnessMin = food.addConstrs((gp.quicksum(hardness[oil]*consume[month, oil] for oil in oils)
                 >= min_hardness*produce[month] for month in months), "Hardness_lower")

HardnessMax = food.addConstrs((gp.quicksum(hardness[oil]*consume[month, oil] for oil in oils)
                 <= max_hardness*produce[month] for month in months), "Hardness_upper")
"""

#As duas restrições são uma lista de restrições para cada mês

# i = month
# j = oil

#primeira igualdade
print("         y_i*(b_i - sumatorio)         y_i*(Slack)")
for month in months:

  y_i = HardnessMin[month].Pi
  
  b_i = min_hardness*produce[month].X

  sumatorio = gp.quicksum(hardness[oil]*consume[month, oil].X for oil in oils).getValue()

  resultado = y_i*(b_i - sumatorio)

  print("i:", oils.index(oil), "     = ", resultado, "                     = ", y_i*HardnessMin[month].Slack)


print("         y_i*(b_i - sumatorio)         y_i*(Slack)")
for month in months:

  y_i = HardnessMax[month].Pi
  
  b_i = max_hardness*produce[month].X

  sumatorio = gp.quicksum(hardness[oil]*consume[month, oil].X for oil in oils).getValue()

  resultado = y_i*(b_i - sumatorio)

  print("i:", oils.index(oil), "     = ", resultado, "                     = ", y_i*HardnessMax[month].Slack)

         y_i*(b_i - sumatorio)         y_i*(Slack)
i: 4      =  -0.0                      =  -0.0
i: 4      =  -0.0                      =  -0.0
i: 4      =  -0.0                      =  -0.0
i: 4      =  -0.0                      =  -0.0
i: 4      =  -0.0                      =  -0.0
i: 4      =  -0.0                      =  -0.0
         y_i*(b_i - sumatorio)         y_i*(Slack)
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0


In [234]:
"""
#6. Mass Conservation
MassConservation = food.addConstrs((consume.sum(month) == produce[month] for month in months), "Mass_conservation")
"""

#Uma restrição para cada mês

# i = month
# j = oil

#primeira igualdade
print("         y_i*(b_i - sumatorio)         y_i*(Slack)")
for month in months:

  y_i = MassConservation[month].Pi
  
  b_i = produce[month].X

  sumatorio = consume.sum(month).getValue()

  resultado = y_i*(b_i - sumatorio)

  print("i:", oils.index(oil), "     = ", resultado, "                     = ", y_i*MassConservation[month].Slack)

         y_i*(b_i - sumatorio)         y_i*(Slack)
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
i: 4      =  0.0                      =  0.0
